Plot keystone figures

In [11]:
import os 
import sys
#id pheno
indir = "../result/large_scale_cohort"
leave_path = "../result/GCN_fix_tree/leaves_cluster.tsv"
outdir = "../result/keystone"
if not os.path.exists(outdir):
    os.makedirs(outdir)
plist = ['ACVD',
 'BD',
 'CRC',
 'IBD',
 'IGT',
 'CFS',
 'STH',
 'T2D',
 'adenoma',
 'asthma',
 'carcinoma_surgery_history',
 'hypertension',
 'migraine',
 'schizofrenia']    


In [ ]:
for disease in plist:
    dir_path = os.path.join(indir, disease)
    for id in os.listdir(dir_path):
        cohort_dir = os.path.join(dir_path, id)
        if not os.path.isdir(cohort_dir):
            continue
        e = os.system("perl 1.mk_eigen_stat.pl {} {} {} {} {} {}".format(id, disease, indir, indir, leave_path, outdir))


In [12]:
# check if Rscript exists
python_executable = sys.executable
python_dir = os.path.dirname(python_executable)
rscript_path = os.path.join(python_dir, 'Rscript')
if os.path.exists(rscript_path):
    print(f"Rscript path: {rscript_path}")
else:
    print(f"Could not find Rscript in Python directory: {rscript_path}")
    print("Please ensure R is installed in your conda environment")

Rscript path: /home/chelijia/anaconda3/envs/meta_fr_r/bin/Rscript


In [16]:
for disease in plist:
    dir_path = os.path.join(indir, disease)
    for id in os.listdir(dir_path):
        cohort_dir = os.path.join(dir_path, id)
        if not os.path.isdir(cohort_dir):
            continue
        prefix = "{}/{}".format(outdir, id)
        e = os.system("Rscript page_rank.2.R {} {}".format(prefix, id))
        